# 题目

给定 N 个位置，机器人的初始位置 S，机器人的目标位置 A，以及步数 K。机器人每一步必须在可动方向移动一格。

计算：机器人在规定步数到达目标位置的走法有多少。

In [5]:
import time

def robot(N: int, start:int, aim: int, K: int) -> int:
    return process1(start, K, aim, N)

def process1(cur: int, rest:int, aim: int, N:int) -> int:
    """
    机器人当前来到的位置是cur
    机器人还有rest步需要走
    最终目标是aim
    位置有 1 ~ N
    返回：机器人从cur出发走过rest步之后最终停在aim的方法数
    """
    if rest == 0: return 1 if cur == aim else 0
    if cur == 1: return process1(2, rest - 1, aim, N)
    if cur == N: return process1(N - 1, rest - 1, aim, N)
    return process1(cur - 1, rest - 1, aim, N) + process1(cur + 1, rest - 1, aim, N)
    
start = time.time()
print(robot(19, 2, 8, 26))
end = time.time()
print("耗时：", end-start)

3749460
耗时： 13.017210245132446


## 优化

In [120]:
def robot2(N: int, start: int, aim: int, K: int) -> int:
    dp = []
    for i in range(N):
        dp2=[]
        for j in range(K+1):
            dp2.append(-1)
        dp.append(dp2)
    return process2(start, K, aim, N, dp)

def process2(cur: int, rest:int, aim: int, N:int, dp: list) -> int:
    """
    cur 的范围是 1 ~ N
    rest 的范围是 0 ~ K

    Args:
        cur (int): 当前位置
        rest (int): 剩余步数
        aim (int): 目标位置
        N (int): 位置总数
        dp (list): 缓存表

    Returns:
        int: 达到目的的方法总数
    """
    if dp[cur - 1][rest] != -1: return dp[cur - 1][rest]
    ans = 0
    if rest == 0: ans = 1 if cur == aim else 0
    elif cur == 1: ans = process2(2, rest - 1, aim, N, dp)
    elif cur == N: ans = process2(N - 1, rest - 1, aim, N, dp)
    else: ans = process2(cur - 1, rest - 1, aim, N, dp) + process2(cur + 1, rest - 1, aim, N, dp) 
    dp[cur-1][rest] = ans
    return ans

start = time.time()
print(robot2(19, 2, 8, 2222))
end = time.time()
print("耗时：", end-start)

50506466906444467888348236079040882518613951749544402488636573566509137853086174414228717304457988115272210413985117839319049649109408465914494017349231618288623909895937580472750217844889462671199633904576442633867068693667497335545354531275037948512663154830323631728844914324093782708896371310933534085935350833057648171696846631266613153078360651093000212476922319863879947447982477388711647467814025683042295583324080946362023288420538004327444066505999252676901595032918878827068437996821666215967743880782541821423219105073584302994934866633904849579466743611144019347756973315063399724585529436731427308758946549996043736095515237473985809284449982
耗时： 0.052857160568237305


## 二次优化

简化例子：
总共有 **5** 个位置 （1 ~ 5）
出发位置为 **2**
目标位置为 **4**
步数为 **6**

将缓存表画出来，cur不会在0的位置

|rest ==|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|cur == 0|X|X|X|X|X|X|X|
|cur == 1||||||
|cur == 2||||||
|cur == 3||||||
|cur == 4||||||
|cur == 5||||||

我们知道，当 rest == 0 时，只有 cur == aim 时值为1，不然则为0

|rest ==|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|cur == 0|X|X|X|X|X|X|X|
|cur == 1|0|||||
|cur == 2|0|||||
|cur == 3|0|||||
|cur == 4|1|||||
|cur == 5|0|||||

我们知道

* 当 cur == 0 时，步数依赖于 其**左下**格的步数 --> *process1(2, rest - 1, aim, N)*
* 当 cur == N 时，步数依赖于 其**左上**格的步数 --> *process1(N - 1, rest - 1, aim, N)*

|rest ==|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|cur == 0|X|X|X|X|X|X|X|
|cur == 1|0|0|?|??|???|????|?????|
|cur == 2|0|?|??|???|????|?????||
|cur == 3|0|||||
|cur == 4|1|!|!!|!!!|!!!!|!!!!!||
|cur == 5|0|1|!|!!|!!!|!!!!|!!!!!|

* 其余状态时，步数依赖于 其**左上**和**左下**格的步数之和 --> *process1(cur - 1, rest - 1, aim, N) + process1(cur + 1, rest - 1, aim, N)*
* 这是一个变体杨辉三角

|rest ==|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|cur == 0|X|X|X|X|X|X|X|
|cur == 1|0|0|0|**1**|0|**4**|0|
|cur == 2|0|0+0=0|0+**1**=**1**|0|**4**|0|**13**|
|cur == 3|0|0+**1**=**1**|0|**3**|0|**9**|0|
|cur == 4|**1**|0+0=0|**1**+**1**=**2**|0|**5**|0|**14**|
|cur == 5|0|**1**|0|**2**|0|**5**|0|

根据题目，我们的出发位置是2，步数是6，所以答案是**13**

|rest ==|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|cur == 0|X|X|X|X|X|X|X|
|cur == 1|0|0|0|1|0|4|0|
|cur == 2|0|0|1|0|4|0|**13**|
|cur == 3|0|1|0|3|0|9|0|
|cur == 4|1|0|2|0|5|0|14|
|cur == 5|0|1|0|2|0|5|0|

In [140]:
def robot3(N: int, start: int, aim: int, K: int) -> int:
    if N < 2 or start < 1 or start >N or aim < 1 or aim > N or K < 0:
        return -1
    dp = []
    for i in range(N+1):
        dp2=[]
        for j in range(K+1):
            dp2.append(1) if i == aim and j == 0 else dp2.append(0)
        dp.append(dp2)
    for r in range(K+1)[1:]:
        dp[1][r] = dp[2][r-1]
        for cur in range(N)[2:]:
            dp[cur][r] = dp[cur-1][r-1] + dp[cur+1][r-1]
        dp[N][r] = dp[N-1][r-1]
    return dp[start][K]
        

start = time.time()
print(robot3(19, 2, 8, 4444))
end = time.time()
print("耗时：", end-start)

4339855737146802435661299958796839729066875520615581055239187969993179386231179527844737766389897458285258526531052760846754725630957938527486252029239320172424181328157897355660116070774888311658449329606161514823423229480362820631398951902901475586620173711117680876045562428179795334371789901190844101769834387869577159026451759454344442887912321857143777554482955062940210273136524817572638795587870462542082411913127824839741674086075196337509561862852927721698716432941882193045627282437993180548211513361017156541091442496902743637290906321533332017879197157141975037952130145109850420378889425880531475718312558987552015724510944031495121471730742455772971646839672213075980466341961292888209229290885386158208615990844995081745220914855874381795827415388885311753071250384825149838712528617082413093640326521467800084229139491303098932699660833782453667748848489396487175466632009621679710398055666931079972028192656967528752042750313009148844601886605904921972142833064396350778526686055887

In [71]:
n = 6
for i in range(n)[1:]:
    print(i)

1
2
3
4
5
